In [1]:
abstract type AbstractNode end

In [2]:
abstract type LeafNode <: AbstractNode end

In [3]:
mutable struct Variable{T} <: LeafNode
    value::T
    grad::T

    Variable(val::T) where T = new{T}(val)
    Variable(val::T, grad::T) where T = new{T}(val)
end

In [4]:
struct Node{FT <: Function, ArgsT <: Tuple, KwargsT <: NamedTuple} <: AbstractNode
    f::FT
    args::ArgsT
    kwargs::KwargsT
end

In [ ]:
import Pkg; Pkg.add("YAAD")

In [16]:
abstract type Operator end

module Trait
import YAAD: Operator

struct Method{FT} <: Operator
    f::FT
end

struct Broadcasted{FT} <: Operator
    f::FT
end
end # Trait

Main.Trait

In [17]:
struct Node{FT <: Operator, ArgsT <: Tuple, KwargsT <: NamedTuple} <: AbstractNode
    f::FT
    args::ArgsT
    kwargs::KwargsT
end

LoadError: [91minvalid redefinition of constant Node[39m

In [18]:
# wrap function to Method
Node(f::Function, args, kwargs) = Node(Trait.Method(f), args, kwargs)
Node(op, args) = Node(op, args, NamedTuple())

Node

In [19]:
mutable struct CachedNode{NT <: AbstractNode, OutT} <: AbstractNode
    node::NT
    output::OutT
end

In [20]:
function CachedNode(f, args...; kwargs...)
    node = Node(f, args, kwargs.data) # this constructs a Node
    output = forward(node)
    CachedNode(node, output)
end

CachedNode

In [21]:
forward(node::Node) = forward(node.f, map(forward, node.args)...; map(forward, node.kwargs)...)

forward (generic function with 1 method)

In [22]:
struct Linear <: Operator
  w::Matrix{Float64}
  b::Vector{Float64}
end

forward(op::Linear, x::Vector{Float64}) = op.w * x + b

forward (generic function with 2 methods)

In [23]:
forward(node::CachedNode) = (node.output = forward(node.node))

forward (generic function with 3 methods)

In [25]:
function forward(::Method{typeof(sin)}, x)
  sin(x)
end

LoadError: [91mTypeError: in Type{...} expression, expected UnionAll, got Type{Method}[39m

In [26]:
forward(op::Operator, args...; kwargs...) = op.f(args...; kwargs...)

forward (generic function with 4 methods)

In [27]:
(op::Linear)(x::Vector) = op.w * x + b

Nie wiem jak to działa sorry

In [4]:
import Pkg; Pkg.add("SpecialFunctions")
import Pkg; Pkg.add("MacroTools")
include("utils.jl");

  Resolving package versions...
Updating `~/.julia/environments/v1.5/Project.toml`
  [276daf66] + SpecialFunctions v0.10.3
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [5]:
y = :(3x^2 + (2x + 1))

:(3 * x ^ 2 + (2x + 1))

In [6]:
wy = Wengert(y)

Wengert List
y1 = x ^ 2
y2 = 3 * y1
y3 = 2x
y4 = y3 + 1
y5 = y2 + y4


In [7]:
Expr(wy)

:((3 * x ^ 2 + (2x + 1),;))

In [8]:
w = Wengert()
tmp = push!(w, :(x^2))
w

Wengert List
y1 = x ^ 2


In [9]:
push!(w, :($tmp + 1))
w

Wengert List
y1 = x ^ 2
y2 = y1 + 1


In [1]:
struct Dual{T<:Real} <: Real
  x::T
  ϵ::T
end

In [1]:
f(x) = [x[1] * x[2], cos(x[1])]

x = [2, 3]
y = f(x)

2-element Array{Float64,1}:
  6.0
 -0.4161468365471424

Symbolic Differentiation

In [4]:
function derive(ex, x)
  ex == x ? 1 :
  ex isa Union{Number,Symbol} ? 0 :
  error("$ex is not differentiable")
end

derive (generic function with 2 methods)

In [8]:
using MacroTools

y = :(x + 1)

:(x + 1)

In [9]:
@capture(y, a_ * b_)

false

In [11]:
@capture(y, a_ + b_)

true

: what it's mean 
https://docs.julialang.org/en/v1/manual/metaprogramming/

In [12]:
function derive(ex, x)
  ex == x ? 1 :
  ex isa Union{Number,Symbol} ? 0 :
  @capture(ex, a_ + b_) ? :($(derive(a, x)) + $(derive(b, x))) :
  error("$ex is not differentiable")
end

derive (generic function with 2 methods)

In [13]:
y = :(x + 1)
derive(y, :x)

:(1 + 0)

In [14]:
y = :(x + (1 + (x + 1)))
derive(y, :x)

:(1 + (0 + (1 + 0)))

In [15]:
function derive(ex, x)
  ex == x ? 1 :
  ex isa Union{Number,Symbol} ? 0 :
  @capture(ex, a_ + b_) ? :($(derive(a, x)) + $(derive(b, x))) :
  @capture(ex, a_ * b_) ? :($a * $(derive(b, x)) + $b * $(derive(a, x))) :
  @capture(ex, a_^n_Number) ? :($(derive(a, x)) * ($n * $a^$(n-1))) :
  @capture(ex, a_ / b_) ? :($b * $(derive(a, x)) - $a * $(derive(b, x)) / $b^2) :
  error("$ex is not differentiable")
end

derive (generic function with 2 methods)

In [16]:
y = :(3x^2 + (2x + 1))
dy = derive(y, :x)

:((3 * (1 * (2 * x ^ 1)) + x ^ 2 * 0) + ((2 * 1 + x * 0) + 0))

In [17]:
addm(a, b) = a == 0 ? b : b == 0 ? a : :($a + $b)
mulm(a, b) = 0 in (a, b) ? 0 : a == 1 ? b : b == 1 ? a : :($a * $b)
mulm(a, b, c...) = mulm(mulm(a, b), c...)

mulm (generic function with 2 methods)

In [18]:
function derive(ex, x)
  ex == x ? 1 :
  ex isa Union{Number,Symbol} ? 0 :
  @capture(ex, a_ + b_) ? addm(derive(a, x), derive(b, x)) :
  @capture(ex, a_ * b_) ? addm(mulm(a, derive(b, x)), mulm(b, derive(a, x))) :
  @capture(ex, a_^n_Number) ? mulm(derive(a, x),n,:($a^$(n-1))) :
  @capture(ex, a_ / b_) ? :($(mulm(b, derive(a, x))) - $(mulm(a, derive(b, x))) / $b^2) :
  error("$ex is not differentiable")
end

derive (generic function with 2 methods)

In [19]:
y = :(3x^2 + (2x + 1))
dy = derive(y, :x)

:(3 * (2 * x ^ 1) + 2)

In [20]:
ddy = derive(dy, :x)

:(3 * (2 * x ^ 0))

Forward Mode

In [2]:
include("utils.jl");

In [3]:
y = Wengert(:(5sin(log(x))))
derive(y, :x)

Wengert List
y1 = log(x)
y2 = sin(y1)
y3 = 5 * y2
y4 = 1 / x
y5 = cos(y1)
y6 = y5 * y4
y7 = 5 * y6


In [21]:
struct Dual{T<:Real} <: Real
  x::T
  ϵ::T
end

In [22]:
Base.show(io::IO, d::Dual) = print(io, d.x, " + ", d.ϵ, "ϵ")
Dual(1,2)

1 + 2ϵ

In [23]:
import Base: +, -, *, /
a::Dual + b::Dual = Dual(a.x + b.x, a.ϵ + b.ϵ)
a::Dual - b::Dual = Dual(a.x - b.x, a.ϵ - b.ϵ)
a::Dual * b::Dual = Dual(a.x * b.x, b.x * a.ϵ + a.x * b.ϵ)
a::Dual / b::Dual = Dual(a.x * b.x, b.x * a.ϵ - a.x * b.ϵ)

Base.sin(d::Dual) = Dual(sin(d.x), d.ϵ * cos(d.x))
Base.cos(d::Dual) = Dual(cos(d.x), - d.ϵ * sin(d.x))

In [24]:
Base.convert(::Type{Dual{T}}, x::Dual) where T = Dual(convert(T, x.x), convert(T, x.ϵ))
Base.convert(::Type{Dual{T}}, x::Real) where T = Dual(convert(T, x), zero(T))
Base.promote_rule(::Type{Dual{T}}, ::Type{R}) where {T,R} = Dual{promote_type(T,R)}

Dual(1, 2) * 3

3 + 6ϵ

In [25]:
f(x) = x / (1 + x*x)

f(Dual(5., 1.))

130.0 + -24.0ϵ

In [26]:
D(f, x) = f(Dual(x, one(x))).ϵ

D(f, 5.)

-24.0